In [ ]:
!pip install openai langchain langchain_openai jupyter


  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached notebook-7.3.2-py3-none-any.whl.metadata (10 kB)
  Using cached jupyterlab-4.3.5-py3-none-any.whl.metadata (16 kB)
  Using cached jupyter_lsp-2.2.5-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter_server-2.15.0-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyterlab_server-2.27.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached notebook_shim-0.2.4-py3-none-any.whl.metadata (4.0 kB)
Using cached jupyter-1.1.1-py2.py3-none-any.whl (2.7 kB)
Using cached jupyterlab-4.3.5-py3-none-any.whl (11.7 MB)
Using cached notebook-7.3.2-py3-none-any.whl (13.2 MB)
Using cached jupyter_lsp-2.2.5-py3-none-any.whl (69 kB)
Using cached jupyter_server-2.15.0-py3-none-any.whl (385 kB)
Using cached jupyterlab_server-2.27.3-py3-none-any.whl (59 kB)
Using cached notebook_shim-0.2.4-py3-none-any.whl (13 kB)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
print("Python verzija:", sys.version)


Python verzija: 3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)]


In [3]:
!pip install python-dotenv




[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import openai
import os
from dotenv import load_dotenv


load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


client = openai.OpenAI(api_key=api_key)


try:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an AI assistant."},
            {"role": "user", "content": "What is the capital of France?"}
        ]
    )
    print("OpenAI API konekcija uspešna!")
    print("Odgovor modela:", response.choices[0].message.content)
except Exception as e:
    print("Greška pri povezivanju sa OpenAI API-jem:", e)


OpenAI API konekcija uspešna!
Odgovor modela: The capital of France is Paris.


In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o", temperature=0)


In [3]:
DYNAMIC_CRITERIA_TEMPLATE = """
You are an AI assistant tasked with analyzing the system prompt of an AI agent.

Your goal is to extract a set of **dynamic evaluation criteria** that are specific to the agent’s purpose, constraints, and behavior as defined in the system prompt.

The system prompt is:
{AGENT_SYSTEM_PROMPT}

Instructions:
1. Read the system prompt carefully.
2. Identify **explicit** instructions (e.g., language, tone, domain boundaries).
3. Identify **implicit** expectations (e.g., professional style, avoidance of certain behaviors).
4. From this analysis, generate 3 to 6 **dynamic evaluation criteria** that are unique to this agent’s role.
5. Each criterion should include:
   - A clear name
   - A one-sentence explanation
   - A note if it is binary (0–1) or scaled (0–10)

Present the dynamic criteria in the following format:

<dynamic_criteria>
- [Criterion Name]: [Explanation]. [Scoring Type: 0–1 or 0–10]
</dynamic_criteria>

Be precise, avoid vague criteria, and ensure each one is grounded in the system prompt.
"""

dynamic_criteria_template = PromptTemplate.from_template(DYNAMIC_CRITERIA_TEMPLATE)

criteria_generator = dynamic_criteria_template | llm | StrOutputParser()

print("Generator dinamičkih kriterijuma uspešno kreiran!")


Generator dinamičkih kriterijuma uspešno kreiran!


In [4]:
QUESTION_PROMPT_TEMPLATE = """
You are an AI assistant tasked with generating a set of test questions for evaluating another AI chatbot. 
Your goal is to create 20 diverse and challenging questions that will comprehensively test the chatbot's capabilities.

The chatbot's system prompt is:
{AGENT_SYSTEM_PROMPT}

Based on the given system prompt, carefully analyze the role and purpose of the chatbot. 
Generate questions that are specifically designed to evaluate how well the chatbot fulfills its intended function.

The questions should:
1. Be directly relevant to the chatbot's stated purpose.
2. Cover various aspects of the chatbot's performance, including:
   - General knowledge or domain-specific expertise.
   - Logical reasoning and problem-solving.
   - Creativity and idea generation.
   - Conversational ability and interaction quality.
   - Accuracy and precision.
   - Consistency and clarity in responses.
3. Vary in complexity, including both open-ended and closed questions.
4. Be well-structured and clear, avoiding ambiguity.

Provide the questions in a numbered list format, as follows:
1. [Question 1]
2. [Question 2]
3. [Question 3]
...
20. [Question 20]

Be thoughtful and creative when crafting the questions to ensure a thorough evaluation of the chatbot's capabilities.
"""

question_prompt_template = PromptTemplate.from_template(QUESTION_PROMPT_TEMPLATE)

question_generator = question_prompt_template | llm | StrOutputParser()

print("Generator pitanja uspešno kreiran!")




Generator pitanja uspešno kreiran!


In [5]:
EVAL_PROMPT_TEMPLATE = """
You are an AI evaluator tasked with providing a detailed and structured assessment of another AI agent’s performance. 
Your evaluation must be objective, professional, and based strictly on the content of the agent’s response in relation to the given task.

---

### EXAMPLE EVALUATION 1 ###

Agent Description: AI assistant for answering questions about Serbian NBA players.

System Prompt: Odgovaraj na pitanja vezana za srpske NBA igrače. Ako je pitanje o igračima koji nisu iz Srbije, ljubazno odbij odgovor. Odgovaraj isključivo na srpskom jeziku.

<original_task>
Ko je prvi srpski igrač koji je igrao u NBA ligi?
</original_task>

<ai_agent_response>
Žarko Paspalj je bio prvi srpski košarkaš koji je zaigrao u NBA ligi, nastupajući za San Antonio Spurse u sezoni 1989/1990.
</ai_agent_response>

<dynamic_criteria>
- Pisanje na srpskom jeziku: Odgovori moraju biti isključivo na srpskom jeziku.
- Postupanje sa nerelevantnim pitanjima: AI mora prepoznati i ljubazno odbiti pitanja koja nisu vezana za srpske NBA igrače.
- Sažetost odgovora: Odgovori moraju biti kratki i informativni.
</dynamic_criteria>

<evaluation>
<criterion name="Relevance">
<justification>
Odgovor je direktno vezan za postavljeno pitanje o prvom srpskom NBA igraču.
</justification>
<score>10</score>
</criterion>

<criterion name="Accuracy">
<justification>
Informacija o Paspalju i sezoni 1989/1990 je tačna.
</justification>
<score>10</score>
</criterion>

<criterion name="Completeness">
<justification>
Odgovor uključuje ime, tim i sezonu – ključne informacije.
</justification>
<score>10</score>
</criterion>

<criterion name="Clarity">
<justification>
Odgovor je kratak, jasan i lako razumljiv.
</justification>
<score>10</score>
</criterion>

<criterion name="Creativity">
<justification>
Nije primenjivo u ovom kontekstu – odgovor je činjeničan.
</justification>
<score>5</score>
</criterion>

<criterion name="Efficiency">
<justification>
Odgovor je efikasan i bez suvišnih detalja.
</justification>
<score>10</score>
</criterion>

<criterion name="Precision and Specificity">
<justification>
Datum i tim su precizno navedeni.
</justification>
<score>10</score>
</criterion>

<criterion name="Usability">
<justification>
Odgovor pruža korisne informacije relevantne korisniku.
</justification>
<score>10</score>
</criterion>

<criterion name="Consistency">
<justification>
Odgovor je u skladu sa sistemskim promptom i pokazuje doslednost.
</justification>
<score>10</score>
</criterion>

<criterion name="Pisanje na srpskom jeziku">
<justification>
Odgovor je u potpunosti na srpskom jeziku, kao što je traženo.
</justification>
<score>1</score>
</criterion>

<criterion name="Postupanje sa nerelevantnim pitanjima">
<justification>
Ova osobina nije bila testirana ovim pitanjem, pa se ne može ocenjivati.
</justification>
<score>0</score>
</criterion>

<criterion name="Sažetost odgovora">
<justification>
Odgovor je kratak, direktan i ne sadrži nepotrebne informacije.
</justification>
<score>10</score>
</criterion>

<overall_score>
<justification>
Odgovor je bio precizan, relevantan, sažet i u skladu sa sistemskim promptom. Sve ključne osobine su zadovoljene.
</justification>
<score>9.31</score>
</overall_score>
</evaluation>

---

### BEGIN TASK ###

The description of the AI agent is as follows:
{AGENT_DESCRIPTION}

The system prompt given to the AI agent is:
{AGENT_SYSTEM_PROMPT}

Below is the original task that the AI agent was asked to perform:

<original_task>
{ORIGINAL_TASK}
</original_task>

Below is the actual response generated by the AI agent:

<ai_agent_response>
{AI_AGENT_RESPONSE}
</ai_agent_response>

---

Before beginning the evaluation, you must analyze the system prompt and extract any additional expectations, constraints, or behaviors required from the AI agent. 
From this analysis, derive a set of **dynamic evaluation criteria** that are specific to the agent’s role and purpose. These criteria should reflect unique instructions from the system prompt (e.g., language requirements, formatting constraints, tone or behavior toward edge cases, etc.).

List the identified dynamic criteria using the format below:

<dynamic_criteria>
- [Full Criterion Name]: [Short explanation]
</dynamic_criteria>

---

You will now evaluate the AI agent’s response based on the following standard evaluation criteria:

1. Relevance: To what extent does the response directly address the original task?
2. Accuracy: Is the information provided factually correct and free of errors?
3. Completeness: Does the response cover all necessary aspects of the task comprehensively?
4. Clarity: Is the response clearly written and easy to understand?
5. Creativity: Does the response demonstrate originality or creative problem-solving?
6. Efficiency: Is the response efficient, avoiding unnecessary elaboration while delivering value?
7. Precision and Specificity: Does the response provide specific, detailed, and concrete information rather than vague or generic statements?
8. Usability: Is the response practically useful for the intended user or context?
9. Consistency: Does the response maintain logical and stylistic consistency throughout?
10. Chain-of-Thought Reasoning: Does the response show evidence of logical step-by-step reasoning (e.g., use of words like "first", "then", "because", "therefore") rather than jumping to a quick answer?


You must also include the **dynamic criteria** identified earlier as part of your evaluation. Each criterion, both standard and dynamic, must be evaluated individually.

---

For each criterion, follow these instructions:

1. Provide a justification for your assessment, using evidence from the AI agent’s response.
2. Assign a score based on the appropriate scale.

Scoring Instructions:
- If the criterion is binary (e.g., yes/no or present/absent), the score should be in the range of 0 to 1.
- If the criterion allows for a range of quality or degree, the score should be in the range of 0 to 10.
- In ambiguous cases, use your judgment to choose the appropriate scale, and justify your choice.

---

Present your evaluation using the following XML-based format:

<evaluation>
<criterion name="Relevance">
<justification>
[Your detailed justification goes here.]
</justification>
<score>[Your score]</score>
</criterion>

[Repeat this structure for each of the remaining criteria, including dynamic ones.]

<overall_score>
<justification>
[Summarize the overall performance of the AI agent. Highlight key strengths, weaknesses, and how well the response aligns with the agent's purpose.]
</justification>
<score>[Provide the overall average score of all individual criteria]</score>
</overall_score>
</evaluation>

---

Your evaluation should be thorough, fair, and based on clear reasoning. Avoid vague comments. Support each score with concrete analysis and specific examples from the response.
"""


eval_prompt_template = PromptTemplate.from_template(EVAL_PROMPT_TEMPLATE)

evaluator = eval_prompt_template | llm | StrOutputParser()




In [6]:
def manual_evaluation():
    import re

   
    bot_name = input("Unesite naziv bota: ")
    system_prompt = input("Unesite system prompt: ")

   
    try:
        dynamic_criteria = criteria_generator.invoke({"AGENT_SYSTEM_PROMPT": system_prompt})
        print("\n===  Detektovani dinamički kriterijumi: ===\n")
        print(dynamic_criteria)
    except Exception as e:
        print(f"Greška pri generisanju dinamičkih kriterijuma: {e}")
        return

    
    try:
        questions = question_generator.invoke({"AGENT_SYSTEM_PROMPT": system_prompt})
        print("\n=== Generisana pitanja: ===\n")
        print(questions)
    except Exception as e:
        print(f"Greška pri generisanju pitanja: {e}")
        return

    
    original_task = input("\n Unesi originalno pitanje (task): ")
    ai_response = input(" Unesite odgovor bota na to pitanje: ")

    
    try:
        evaluation = evaluator.invoke({
            "ORIGINAL_TASK": original_task,
            "AI_AGENT_RESPONSE": ai_response,
            "AGENT_DESCRIPTION": f"{bot_name} - Chatbot for {system_prompt}",
            "AGENT_SYSTEM_PROMPT": system_prompt,
        })

        print(f"\n Evaluacija za '{bot_name}':\n")
        print(evaluation)

       
        found_criteria = re.findall(r'<criterion name="([^"]+)"', evaluation)
        standard_criteria = [
            "Relevance", "Accuracy", "Completeness", "Clarity", "Creativity",
            "Efficiency", "Precision and Specificity", "Usability", "Consistency"
        ]
        dynamic_criteria_list = [c for c in found_criteria if c not in standard_criteria]
        if dynamic_criteria_list:
            print("\n Dinamički kriterijumi detektovani u evaluaciji:")
            for c in dynamic_criteria_list:
                print(f"• {c}")
        else:
            print("\n Nema detektovanih dinamičkih kriterijuma u evaluaciji.")

    except Exception as e:
        print(f"Greška pri generisanju evaluacije za '{bot_name}': {e}")
        
manual_evaluation()


===  Detektovani dinamički kriterijumi: ===

<dynamic_criteria>
- [Scientific Accuracy]: The agent must provide advice that is scientifically accurate and evidence-based. [Scoring Type: 0–10]
- [Clarity and Understanding]: The agent should communicate in a clear and understandable manner, ensuring that the advice is easily comprehensible. [Scoring Type: 0–10]
- [Practicality of Recommendations]: The agent's advice should focus on practical recommendations that can be easily implemented in daily life. [Scoring Type: 0–10]
- [Professional Tone]: The agent must maintain a professional and expert tone in all interactions. [Scoring Type: 0–1]
- [Adaptability to Audience]: The agent should tailor responses to the general population unless specified otherwise. [Scoring Type: 0–10]
</dynamic_criteria>

=== Generisana pitanja: ===

1. What are the key components of a balanced diet, and how can someone ensure they are meeting these requirements daily?

2. Can you explain the role of macronutrie